In [1]:
!nvidia-smi

Wed Nov  8 15:32:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu py7zr rouge_score -q

In [3]:
!pip install --upgrade accelerate
#uninstalled the old transformer and accelerate and install latest one.
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.35.0
Uninstalling transformers-4.35.0:
  Successfully uninstalled transformers-4.35.0
Found existing installation: accelerate 0.24.1
Uninstalling accelerate-0.24.1:
  Successfully uninstalled accelerate-0.24.1
  Using cached transformers-4.35.0-py3-none-any.whl (7.9 MB)
  Using cached accelerate-0.24.1-py3-none-any.whl (261 kB)


In [4]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk, load_metric
from matplotlib import pyplot as plt
import pandas as pd

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
import warnings
warnings.filterwarnings("ignore")

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
#check gpu avaialablity
device= "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [6]:
#transformer base model for text summarization task
model_ckpt= "google/pegasus-cnn_dailymail"

#same model use of tokenizer also
tokenizer= AutoTokenizer.from_pretrained(model_ckpt, )
#get the pegasus model
model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
#download & unzip data

!wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
!unzip summarizer-data.zip

--2023-11-08 15:34:28--  https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip [following]
--2023-11-08 15:34:29--  https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip’

summarizer-data.zip 100%[===================>]   7.54M  --.-KB/s    in 0.09s   

2023-11-08 15:34:29 (84.4 MB/s) - ‘summarizer-data.zip’ saved [7903594/790

In [8]:
#load the dataset
dataset_samsum=load_from_disk("samsum_dataset")
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [9]:
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]

print(f"Split lengths : {split_lengths}")
print(f"features : {dataset_samsum['train'].column_names}")

print("Dialogue \n")

print(dataset_samsum["test"][1]["dialogue"])
print()
print("summary \n")
print(dataset_samsum["test"][1]["summary"])

Split lengths : [14732, 819, 818]
features : ['id', 'dialogue', 'summary']
Dialogue 

Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

summary 

Eric and Rob are going to watch a stand-up on youtube.


In [10]:
def convert_example_to_features(example_batch):
  input_encodings=tokenizer(example_batch["dialogue"], max_length=1024, truncation=True)

  with tokenizer.as_target_tokenizer():
    target_encodings=tokenizer(example_batch["summary"],max_length=128,truncation=True)

  return{
      "input_ids":input_encodings["input_ids"],
      "attention_mask":input_encodings["attention_mask"],
      "labels":target_encodings["input_ids"]
  }

In [11]:
#apply encoding function on dataset
dataset_samsum_pt=dataset_samsum.map(convert_example_to_features,batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [12]:
dataset_samsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [13]:
dataset_samsum_pt["train"][1]

{'id': '13728867',
 'dialogue': 'Olivia: Who are you voting for in this election? \r\nOliver: Liberals as always.\r\nOlivia: Me too!!\r\nOliver: Great',
 'summary': 'Olivia and Olivier are voting for liberals in this election. ',
 'input_ids': [18038,
  151,
  2632,
  127,
  119,
  6228,
  118,
  115,
  136,
  2974,
  152,
  10463,
  151,
  35884,
  130,
  329,
  107,
  18038,
  151,
  2587,
  314,
  1242,
  10463,
  151,
  1509,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [18038, 111, 34296, 127, 6228, 118, 33195, 115, 136, 2974, 107, 1]}

In [14]:
dataset_samsum_pt

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [15]:
#Training
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collactor=DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

In [16]:
from  transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir="pegasus-samsum",num_train_epochs=1,warmup_steps=500,
    per_device_train_batch_size=1,per_device_eval_batch_size=1,
    weight_decay=0.01,logging_steps=10,
    evaluation_strategy="steps", eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
    )

In [17]:
trainer = Trainer(
    model=model_pegasus,args=trainer_args,
    tokenizer = tokenizer,  data_collator=seq2seq_data_collactor,
    train_dataset=dataset_samsum_pt["test"],
    eval_dataset=dataset_samsum_pt["validation"]
)

In [18]:
trainer.train()

You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=51, training_loss=3.0754146295435287, metrics={'train_runtime': 153.8798, 'train_samples_per_second': 5.322, 'train_steps_per_second': 0.331, 'total_flos': 313317832187904.0, 'train_loss': 3.0754146295435287, 'epoch': 1.0})

In [19]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [20]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

In [21]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

100%|██████████| 5/5 [00:17<00:00,  3.41s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.017505,0.0,0.01502,0.014873


In [22]:
#save trained model
model_pegasus.save_pretrained("pegasus_samsum_model")

## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [23]:
#load tokenizer
tokenizer=AutoTokenizer.from_pretrained("/content/tokenizer")

In [24]:
#make prediction:
gen_kwargs  = {"length_penality":0.8, "num_beams":8, "max_lenght":128} #model arguments
pipe=pipeline("summarization",model="pegasus_samsum_model",tokenizer=tokenizer) #prepare prediction pipeline

In [26]:

#take any random dialogue from test dataset and make prediction using prediction pipeline
import numpy as np
text_record=np.random.choice(dataset_samsum["test"])
sample_text=text_record["dialogue"]
reference=text_record["summary"]
print("Dialogue : ",sample_text,"\n")
print("Reference : ",reference)

print("Model Summary : ",pipe(sample_text,gen_kwargs)[0]["summary_text"])

Ignoring args : ({'length_penality': 0.8, 'num_beams': 8, 'max_lenght': 128},)


Dialogue :  Adelina: Hi handsome. Where you you come from?
Cyprien: What do you mean?
Adelina: What do you mean, "what do you mean"? It's a simple question, where do you come from?
Cyprien: Well I was born in Jarrow, live in London now, so you could say I came from either of those places
Cyprien: I was educated in Loughborouogh, so in a sense I came from there.
Adelina: OK. 
Cyprien: In another sense I come from my mother's vagina, but I dare say everyone can say that.
Adelina: Are you all right?
Cyprien: IN another sense I come from the atoms in the air that I breath or the food I eat, which comes to me from many places, so all I can say is "I come from Planet Earth".
Adelina: OK, bye. If you're gonna be a dick...
Cyprien: Wait, what you got against earthlings? 

Reference :  Cyprien irritates Adelina by giving too many responses.
Model Summary :  Cyprien: I was born in Jarrow, live in London now, so you could say I came from either of those places .<n>In another sense I come from my 